# 동적 프로그래밍: 가치 및 정책 반복

이 섹션에서는 다음 기능과 함께 다음 다이어그램에 묘사된 3 x 4 그리드로 구성된 장난감 환경에 값과 정책 반복을 적용합니다.
- **상태**: 2차원 좌표로 표현된 11가지 상태입니다. 한 필드는 액세스할 수 없으며 가장 오른쪽 열의 상위 두 상태는 터미널 상태입니다. 즉, 에피소드가 종료됩니다.- **액션**: 각 단계, 즉 위, 아래, 왼쪽, 오른쪽으로 이동합니다. 환경은 무작위로 지정되어 있어 행동이 의도하지 않은 결과를 초래할 수 있습니다. 각 행동마다 예상된 상태로 이동할 확률은 80%이고, 인접한 방향(예: 위쪽이 아닌 오른쪽 또는 왼쪽, 오른쪽이 아닌 위쪽 또는 아래쪽)으로 이동할 확률은 10%입니다.- **보상**: 오른쪽 패널에 표시된 것처럼 터미널 상태의 +1/-1 보상을 제외하고 각 상태의 결과는 -.02입니다.

<img src="../assets/mdp.png" width="500">

이전 GridWorld 다이어그램의 오른쪽 패널은 Value Iteration과 해당 탐욕 정책에 의해 생성된 최적의 추정 가치를 보여줍니다. 환경의 불확실성과 결합된 부정적인 보상은 부정적인 최종 상태에서 벗어나는 것을 포함하는 최적의 정책을 생성합니다.
결과는 보상과 할인 요인 모두에 민감합니다. 부정적인 상태의 비용은 주변 필드의 정책에 영향을 미치며, 최적의 작업 선택을 변경하는 임계값 수준을 식별하려면 해당 노트북의 예를 수정해야 합니다.

## 가져오기 및 설정

In [1]:
%matplotlib inline

from time import process_time
import numpy as np
import pandas as pd
from mdptoolbox import mdp
from itertools import product

## 그리드월드 설정

### 상태, 행동, 보상

환경 매개변수를 정의하는 것부터 시작하겠습니다.

In [2]:
grid_size = (3, 4)
blocked_cell = (1, 1)
baseline_reward = -0.02
absorbing_cells = {(0, 3): 1, (1, 3): -1}

In [3]:
actions = ['L', 'U', 'R', 'D']
num_actions = len(actions)
probs = [.1, .8, .1, 0]

1차원 표현과 2차원 표현 사이를 변환해야 하는 경우가 자주 있으므로 이 목적을 위해 두 가지 도우미 함수를 정의하겠습니다. 상태는 1차원이고 셀은 해당하는 2차원 좌표입니다.

In [4]:
to_1d = lambda x: np.ravel_multi_index(x, grid_size)
to_2d = lambda x: np.unravel_index(x, grid_size)

또한 코드를 더욱 간결하게 만들기 위해 일부 데이터 포인트를 미리 계산합니다.

In [5]:
num_states = np.product(grid_size)
cells = list(np.ndindex(grid_size))
states = list(range(len(cells)))

In [6]:
cell_state = dict(zip(cells, states))
state_cell= dict(zip(states, cells))

In [7]:
absorbing_states = {to_1d(s):r for s, r in absorbing_cells.items()}
blocked_state = to_1d(blocked_cell)

우리는 각 상태에 대한 보상을 저장합니다.

In [8]:
state_rewards = np.full(num_states, baseline_reward)
state_rewards[blocked_state] = 0
for state, reward in absorbing_states.items():
    state_rewards[state] = reward

In [9]:
action_outcomes = {}
for i, action in enumerate(actions):
    probs_ = dict(zip([actions[j % 4] for j in range(i, num_actions + i)], probs))
    action_outcomes[actions[(i + 1) % 4]] = probs_

확률적 환경을 설명하기 위해 주어진 행동에 대한 실제 움직임에 대한 확률 분포도 계산해야 합니다.

In [10]:
action_outcomes

{'U': {'L': 0.1, 'U': 0.8, 'R': 0.1, 'D': 0},
 'R': {'U': 0.1, 'R': 0.8, 'D': 0.1, 'L': 0},
 'D': {'R': 0.1, 'D': 0.8, 'L': 0.1, 'U': 0},
 'L': {'D': 0.1, 'L': 0.8, 'U': 0.1, 'R': 0}}

이제 MDP의 주요 입력인 전환 행렬을 계산할 준비가 되었습니다.

### 전환 매트릭스

전이 행렬은 각 이전 상태 및 동작 A에 대해 특정 상태 S로 끝날 확률을 $P(s^\prime \mid s, a)$로 정의합니다. `pymdptoolbox`을 시연하고 전환 및 보상을 지정하는 데 사용할 수 있는 형식 중 하나를 사용합니다. 두 가지 전환 확률에 대해 $A \times S \times S$ 차원의 `NumPy` 배열을 만듭니다.
먼저, 각 시작 셀에 대한 대상 셀을 계산하고 이동합니다.

In [11]:
def get_new_cell(state, move):
    cell = to_2d(state)
    if actions[move] == 'U':
        return cell[0] - 1, cell[1]
    elif actions[move] == 'D':
        return cell[0] + 1, cell[1]
    elif actions[move] == 'R':
        return cell[0], cell[1] + 1
    elif actions[move] == 'L':
        return cell[0], cell[1] - 1

In [12]:
state_rewards

array([-0.02, -0.02, -0.02,  1.  , -0.02,  0.  , -0.02, -1.  , -0.02,
       -0.02, -0.02, -0.02])

다음 함수는 인수의 시작 `state`, `action` 및 `outcome`을 사용하여 전환 확률과 보상을 채웁니다.

In [13]:
def update_transitions_and_rewards(state, action, outcome):
    if state in absorbing_states.keys() or state == blocked_state:
        transitions[action, state, state] = 1
    else:
        new_cell = get_new_cell(state, outcome)
        p = action_outcomes[actions[action]][actions[outcome]]
        if new_cell not in cells or new_cell == blocked_cell:
            transitions[action, state, state] += p
            rewards[action, state, state] = baseline_reward
        else:
            new_state= to_1d(new_cell)
            transitions[action, state, new_state] = p
            rewards[action, state, new_state] = state_rewards[new_state]

다음과 같이 자리 표시자 데이터 구조를 만들고 $A \times S \times S$의 데카르트 곱을 반복하여 전환 및 보상 값을 생성합니다.

In [14]:
rewards = np.zeros(shape=(num_actions, num_states, num_states))
transitions = np.zeros((num_actions, num_states, num_states))
actions_ = list(range(num_actions))
for action, outcome, state in product(actions_, actions_, states):
    update_transitions_and_rewards(state, action, outcome)

In [15]:
rewards.shape, transitions.shape

((4, 12, 12), (4, 12, 12))

## PyMDP 도구 상자

Q-학습을 포함하여 몇 가지 추가 알고리즘이 포함된 [pymdp도구 상자](https://pymdptoolbox.readthedocs.io/en/latest/api/mdptoolbox.html) Python 라이브러리를 사용하여 MDP를 해결할 수도 있습니다.

### 가치 반복

In [16]:
gamma = .99
epsilon = 1e-5

`ValueIteration`을 실행하려면 `.run()` 메서드를 호출하기 전에 원하는 구성 옵션과 보상 및 전환 행렬을 사용하여 해당 객체를 인스턴스화하세요.

In [17]:
vi = mdp.ValueIteration(transitions=transitions,
                        reward=rewards,
                        discount=gamma,
                        epsilon=epsilon)

vi.run()
f'# Iterations: {vi.iter:,d} | Time: {vi.time:.4f}'

'# Iterations: 31 | Time: 0.0006'

In [18]:
policy = np.asarray([actions[i] for i in vi.policy])
pd.DataFrame(policy.reshape(grid_size))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,L,L


In [19]:
value = np.asarray(vi.V).reshape(grid_size)
pd.DataFrame(value)

,0,1,2,3
0,0.884143,0.925054,0.961986,0.000000
1,0.848181,0.000000,0.714643,0.000000
2,0.808345,0.773328,0.736099,0.516083


### 정책 반복

`PolicyIteration` 함수도 비슷하게 작동합니다.

In [20]:
pi = mdp.PolicyIteration(transitions=transitions,
                        reward=rewards,
                        discount=gamma,
                        max_iter=1000)

pi.run()
f'# Iterations: {pi.iter:,d} | Time: {pi.time:.4f}'

'# Iterations: 7 | Time: 0.0560'

또한 동일한 정책을 생성하지만 가치 함수는 실행마다 다르며 정책이 수렴되기 전에 최적의 값을 달성할 필요가 없습니다.

In [21]:
policy = np.asarray([actions[i] for i in pi.policy])
pd.DataFrame(policy.reshape(grid_size))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,L,L


In [22]:
value = np.asarray(pi.V).reshape(grid_size)
pd.DataFrame(value)

,0,1,2,3
0,0.884143,0.925054,0.961986,-1.389785e-16
1,0.848181,0.000000,0.714643,5.749281e-16
2,0.808345,0.773328,0.736099,5.160828e-01


## 가치 반복

In [23]:
skip_states = list(absorbing_states.keys())+[blocked_state]
states_to_update = [s for s in states if s not in skip_states]

그런 다음 가치 함수를 초기화하고 할인 계수 감마와 수렴 임계값 엡실론을 설정합니다.

In [24]:
V = np.random.rand(num_states)
V[skip_states] = 0

In [25]:
gamma = .99
epsilon = 1e-5

알고리즘은 Bellman 최적성 방정식을 사용하여 가치 함수를 업데이트하고 V의 L1 노름이 절대값으로 엡실론보다 적게 변경될 때 종료됩니다.

In [26]:
iterations = 0
start = process_time()
converged = False
while not converged:
    V_ = np.copy(V)
    for state in states_to_update:
        q_sa = np.sum(transitions[:, state] * (rewards[:, state] + gamma* V), axis=1)
        V[state] = np.max(q_sa)
    if np.sum(np.fabs(V - V_)) < epsilon:
        converged = True

    iterations += 1
    if iterations % 1000 == 0:
        print(np.sum(np.fabs(V - V_)))

f'# Iterations {iterations} | Time {process_time() - start:.4f}'

'# Iterations 24 | Time 0.0023'

### 가치 기능

In [27]:
print(pd.DataFrame(V.reshape(grid_size)))

          0         1         2         3
0  0.884143  0.925054  0.961986  0.000000
1  0.848181  0.000000  0.714643  0.000000
2  0.808345  0.773328  0.736099  0.516083


In [28]:
np.allclose(V.reshape(grid_size), np.asarray(vi.V).reshape(grid_size))

True

### 최적의 정책

In [29]:
for state, reward in absorbing_states.items():
    V[state] = reward

policy = np.argmax(np.sum(transitions * V, 2),0)
policy

array([2, 2, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [30]:
pd.DataFrame(policy.reshape(grid_size)).replace(dict(enumerate(actions)))

,0,1,2,3
0,R,R,R,L
1,U,L,L,L
2,U,L,L,L


## 정책 반복

정책 반복에는 별도의 평가 및 개선 단계가 포함됩니다. 기대 보상과 다음 상태 가치의 합을 최대화하는 행동을 선택하여 개선 부분을 정의합니다. 우리를 최종 상태로 이끄는 행동을 무시하지 않기 위해 일시적으로 최종 상태에 대한 보상을 채웁니다.

In [31]:
def policy_improvement(value, transitions):
    for state, reward in absorbing_states.items():
        value[state] = reward
    return np.argmax(np.sum(transitions * value, 2),0)

In [32]:
V = np.random.rand(num_states)
V[skip_states] = 0
pi = np.random.choice(list(range(num_actions)), size=num_states)

알고리즘은 정책이 안정화될 때까지 탐욕스럽게 선택한 작업에 대한 정책 평가와 정책 개선을 번갈아 수행합니다.

In [33]:
iterations = 0
start = process_time()
converged = False
while not converged:
    pi_ = np.copy(pi)
    for state in states_to_update:
        action = policy[state]
        V[state] = np.dot(transitions[action, state], (rewards[action, state] + gamma* V))
        pi = policy_improvement(V.copy(), transitions)
    if np.array_equal(pi_, pi):
        converged = True
    iterations += 1

f'# Iterations {iterations} | Time {process_time() - start:.4f}'

'# Iterations 3 | Time 0.0009'

정책 반복은 단 세 번의 반복 후에 수렴됩니다. 정책은 알고리즘이 최적의 가치 함수를 찾기 전에 안정화되며 최적의 정책은 약간 다릅니다. 특히 음의 최종 상태 옆에 있는 필드에 대해 더 안전한 왼쪽 대신 위쪽을 제안함으로써 가장 두드러집니다. 이는 예를 들어 여러 라운드의 안정적인 정책을 요구하거나 가치 함수에 대한 임계값을 추가하는 등 수렴 기준을 강화함으로써 방지할 수 있습니다.

In [34]:
pd.DataFrame(pi.reshape(grid_size)).replace(dict(enumerate(actions)))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,L,L


In [35]:
pd.DataFrame(V.reshape(grid_size))

,0,1,2,3
0,0.756333,0.882232,0.933790,0.000000
1,0.683594,0.000000,0.480169,0.000000
2,0.612364,0.552599,0.506767,0.307299
